In [ ]:
import base64
import csv
import pandas as pd
import numpy as np
import pprint
import urllib.request, urllib.parse, urllib.error
import httplib2
from xml.dom import minidom
import getpass

pp = pprint.PrettyPrinter()

fn = "log64_attempts.12-13-2012.csv"

indexes = " OR ".join(["index="+x for x in """directory_services_nonprod
dmv_google_cloud
doh_google_cloud
dol_google_cloud
dtf_http
estreamer
fireeye
hcr_google_cloud
health_datapower
health_network
its_google_cloud
its_okta_nonprod
main
miauditlogs
os
otda_google_cloud
vmware
webny
webny_nonprod
wineventlog""".split("\n")])

earliest_time = "12/12/2021:00:00:00"
latest_time ="-1d@d"
detail = '''Base64 Command "${jndi:*}" | rex "\${jndi.*?Base64\/(?<b64>[^}]*?)}.*" 
| mvexpand b64 | stats values(host) as host earliest(_time) as earliest  by b64 |  mvexpand host 
| convert timeformat="%m/%d/%Y %H:%M:%S" ctime(earliest) as earliest 
| sort host +earliest 
| table host earliest b64'''

query = " ".join(["search earliest=", earliest_time, "latest=", latest_time, indexes, detail])

def d64(x):
    # d(b64) decodes base64 byte string. =Nan if there's an error
    try:
        r = base64.b64decode(x, altchars="-_").decode()
    except:
        r = np.nan
    return r

# Read the splunk extraction into a dataframe
def get_data(fn, method = 0):
    if method == 0:
        return pd.read_csv(fn)

def sprt(events):
    lst = []
    for result in results.ResultsReader(events):
        lst.extend(result.values())
    return lst


In [ ]:
# This job makes a real search and outputs the result.
# Assume an authenticated session is open.

import sys
from time import sleep
import splunklib.results as results
import splunklib.client as client

HOST = "cnsesplunkoperations.svc.ny.gov"
PORT = 8089
USERNAME = "bdk01"
PASSWORD = getpass.getpass()
OWNER = "bdk01"
APP = "search"

# Create a Service instance and log in 
service = client.connect(
    host=HOST,
    port=PORT,
    username=USERNAME,
    password=PASSWORD,
    owner=OWNER,
    app=APP)

# searchquery_normal = "search index=main | stats values(sourcetype)"
searchquery_normal = query
# pp.pprint(query)

kwargs_normalsearch = {
#     "earliest_time": "12/09/2021:00:00:00",
#     "latest_time": "-1d@d",
    "search_mode": "normal"
    }

job = service.jobs.create(searchquery_normal, **kwargs_normalsearch)

# A normal search returns the job's SID right away, so we need to poll for completion
while True:
    while not job.is_ready():
        pass
    stats = {"isDone": job["isDone"],
             "doneProgress": float(job["doneProgress"])*100,
              "scanCount": int(job["scanCount"]),
              "eventCount": int(job["eventCount"]),
              "resultCount": int(job["resultCount"])}

    status = ("\r%(doneProgress)03.1f%%   %(scanCount)d scanned   "
              "%(eventCount)d matched   %(resultCount)d results") % stats

    print(f"{status=}")
    sys.stdout.flush()
    if stats["isDone"] == "1":
        sys.stdout.write("\n\nDone!\n\n")
        break
    sleep(2)

# Get the results and display them
# for result in results.ResultsReader(job.results()):
#     print(result)

save_results = job.results()

def make_list(events):
    lst = []
    for result in results.ResultsReader(events):
        lst.extend(result.values())
    return lst

def make_df(lst, columns):
    n = len(columns)
    ln = len(lst)
    if ln % n != 0:
        return None
    for i in range(0, len(lst), n):
        if not df:
            df = pd.CreateFrame(lst[0::n], columns[0])
        else:
            df[columns[i]] = lst[i::n]
    
    

lst = make_list(save_results)
for i in range(0, len(lst), 3):
    print(f"{lst[i]}  {lst[i+1]}  {d64(lst[i+2])}")

# print(sprt(save_results))
# pp.pprint([x.decode() for x in list(job.results())])
    
# job.cancel()   
# sys.stdout.write('\n')

In [8]:
import pandas as pd
def make_df(lst, column_names):
    # Reshape lst to be a df with n=len(columns_names) columns
    # 0th row has items lst[0] lst[1] ... (n-1)th
    # 1st row has items lst[0+n] lst[1+n] ... lst[(n-1 + n)i
    # ... etc
    num_column = len(column_names)
    len_list = len(lst)
    len_column = len_list//num_column
    df = None
    if len_list % len_column != 0:
        print(f"length of lst ({len_list}) must be even multiple of # of names ({num_column})")
        return df
    import pdb; pdb.set_trace()

    df = pd.DataFrame(lst[0::num_column], columns=list(column_names[col]))
    for col in range(1, num_column):
        df = pd.DataFrame(lst[0::num_column], columns=list(column_names[col]))
    
    return df

lst = [1,2,3,4,5,6,7,8]
cols = ["a", "b"]
make_df(lst, cols)


> /tmp/ipykernel_4558/3292140861.py(15)make_df()
     13         return df
     14     import pdb; pdb.set_trace()
---> 15     for col in range(0, len_list, len_column):
     16         if df:
     17             df[column_names[col]] = lst[col::len_column]

ipdb> c


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [1]:
lst[1::2]

NameError: name 'lst' is not defined

In [ ]:
def main():

    # Get the data
    log = get_data(fn)

    # Decode the b64 column into new column, payload
    log["payload"] = log.loc[:,"b64"].apply(d64)

    # Show the result
    pp.pprint(log[["host", "earliest", "payload"]])

    # Save to csv file
    log[["host", "earliest", "payload"]].to_csv("decode_" + fn)



In [12]:
df=pd.DataFrame([1, 3, 5, 7], columns=["a"])

In [13]:
df


,a
0,1
1,3
2,5
3,7


In [14]:
df["b"] = [2, 4, 6, 8]

In [15]:
df


,a,b
0,1,2
1,3,4
2,5,6
3,7,8


In [ ]:
df.columns = ["a", "b"]

In [ ]:
df


In [16]:
list("a")

['a']